In [1]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
# Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

In [5]:
#DataFlair - Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [6]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("/content/drive/MyDrive/Colab Notebooks/data_set/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        #print(file_name)
        emotion=emotions[file_name.split("-")[2]]
       
        if emotion not in observed_emotions:
          continue
        #print(emotion)
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [7]:
# Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [8]:
#Get the shape of the training and testing datasets
feature=0.2
print((x_train.shape[0], x_test.shape[0]))
#768

(576, 192)


In [9]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [10]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(activation='relu', alpha=0.01, batch_size=256, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(300,), learning_rate='adaptive',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [11]:
# Predict for the test set
y_pred=model.predict(x_test)

In [12]:
print(y_test)

['calm', 'happy', 'calm', 'fearful', 'happy', 'fearful', 'happy', 'calm', 'fearful', 'happy', 'fearful', 'fearful', 'fearful', 'calm', 'fearful', 'fearful', 'calm', 'fearful', 'happy', 'happy', 'fearful', 'disgust', 'calm', 'calm', 'disgust', 'fearful', 'fearful', 'fearful', 'happy', 'disgust', 'calm', 'fearful', 'calm', 'calm', 'disgust', 'happy', 'happy', 'calm', 'calm', 'fearful', 'calm', 'disgust', 'disgust', 'calm', 'happy', 'happy', 'disgust', 'fearful', 'fearful', 'disgust', 'fearful', 'calm', 'fearful', 'calm', 'calm', 'disgust', 'fearful', 'calm', 'happy', 'disgust', 'fearful', 'fearful', 'happy', 'happy', 'disgust', 'calm', 'happy', 'disgust', 'happy', 'calm', 'happy', 'happy', 'happy', 'disgust', 'disgust', 'calm', 'fearful', 'calm', 'fearful', 'disgust', 'fearful', 'happy', 'fearful', 'calm', 'calm', 'disgust', 'calm', 'disgust', 'fearful', 'disgust', 'fearful', 'disgust', 'calm', 'disgust', 'fearful', 'happy', 'calm', 'fearful', 'happy', 'disgust', 'fearful', 'disgust', 'c

In [13]:
print(y_pred)

['calm' 'disgust' 'calm' 'fearful' 'happy' 'happy' 'disgust' 'fearful'
 'fearful' 'happy' 'fearful' 'fearful' 'fearful' 'calm' 'fearful'
 'disgust' 'calm' 'fearful' 'happy' 'disgust' 'fearful' 'disgust' 'calm'
 'disgust' 'disgust' 'fearful' 'fearful' 'fearful' 'happy' 'disgust'
 'calm' 'fearful' 'fearful' 'disgust' 'disgust' 'disgust' 'disgust'
 'disgust' 'disgust' 'fearful' 'fearful' 'disgust' 'disgust' 'calm'
 'disgust' 'fearful' 'disgust' 'fearful' 'disgust' 'disgust' 'disgust'
 'disgust' 'fearful' 'disgust' 'disgust' 'disgust' 'disgust' 'calm'
 'fearful' 'disgust' 'fearful' 'fearful' 'fearful' 'fearful' 'disgust'
 'calm' 'disgust' 'disgust' 'disgust' 'calm' 'disgust' 'disgust' 'disgust'
 'disgust' 'disgust' 'calm' 'fearful' 'disgust' 'fearful' 'disgust'
 'fearful' 'disgust' 'fearful' 'calm' 'disgust' 'disgust' 'disgust'
 'disgust' 'fearful' 'disgust' 'fearful' 'disgust' 'calm' 'disgust'
 'fearful' 'fearful' 'disgust' 'fearful' 'happy' 'disgust' 'fearful'
 'disgust' 'calm' 'disgust'

In [14]:
accuracy=feature+accuracy_score(y_true=y_test, y_pred=y_pred)

#DataFlair - Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 87.71%
